# 14장 합성곱신경망: 컴퓨터비전

#### 감사의 글

자료를 공개한 저자 오렐리앙 제롱과 강의준비에 필요한 자료를 지원한 한빛아카데미에게 진심어린 감사를 전합니다.

* 합성곱신경망(Convolutional Neural Network, CNN)
    * 대뇌의 시각피질 연구에서 출발하여 1980년대부터 이미지인식 분야에서 사용됨.
    * 일부 복잡한 이미지 처리문제에서 사람의 능력 초과함.
    * 주요 활용 예제: 이미지검색 서비스, 자율주행 자동차, 영상 자동분류 등
    * 음성인식, 자연어 처리 등 다양한 분야에서도 활용됨.

## 주요 내용

* CNN의 구성요소

* TF와 케라스를 이용한 CNN의 구현

* 가장 뛰어난 성능의 CNN 구조 살펴보기

* 활용예제
    * 객체탐지(object detection)
        * 이미지 상에서 여러 객체 구분하고 객체 주위에 경계상자(bounding box) 그리기
    * 의미분할(semantic segmentation, 시맨틱 분할)
        * 서로 다른 객체에 속한 픽셀 분류하기

## 시각피질 구조와 합성곱 신경망

* 시각피질 안의 많은 뉴런이 작은 국부수용장(local receptive field)를 가진 것으로 보임.

<img src="images/ch14/homl14-02.png" width="400"/>

<img src="images/ch14/homl14-01.png" width="400"/>

* 합성곱신경망(CNN)으로 발전
    * 합성곱 층(convolution layer)
    * 풀링 층(pooling layer)

<img src="images/ch14/homl14-03.png" width="500"/>

## 합성곱 층(Convolutional layer)

* CNN의 가장 중요한 구성요소

* 사각 형태의 국부수용장을 모방한 CNN 층

<img src="images/ch14/homl14-05.png" width="400"/>

* 층과 제로패딩 사이의 연결

<img src="images/ch14/homl14-06.png" width="400"/>

* 보폭(스트라이드, stride) 2를 사용한 차원축소 지원 CNN 층

<img src="images/ch14/homl14-07.png" width="400"/>

### 필터 (합성곱 커널)

* 뉴런에 사용되는 가중치

* 사용되는 수용장과 동일한 모양의 이미지로 표현됨.

* 다양한 필터 사용
    * 아래 그림은 필터 두 개의 활용을 보여줌.

<img src="images/ch14/homl14-08.png" width="400"/>

### 특성지도(feature map)

* 특성지도: 필터 각각을 사용하여 생성된 출력값

* 필터마다 사용되는 가중치와 편향이 다름.

* 수십, 수백 개의 필터를 사용함.

<img src="images/ch14/homl14-09.png" width="400"/>

## 풀링 층(pooling layer)

* 계산량과 메모리 사용량을 줄이면서 과대적합의 위험도를 줄여주는 용도로 사용됨.

* 가중치 사용하지 않지만, 보폭(stride)를 사용하여 차원을 축소시키는 기능 수행.

### 최대 풀링층(max pooling layer)

#### 예제

<img src="images/ch14/homl14-10.png" width="500"/>

* $2 \times 2$ 크기의 풀링 커널(pooling kernel)
    * 네 개의 셀에 속한 값들 중에서 가장 큰 값만 상위 층으로 전달됨.
* 보폭(stride) 2
    * 하위 입력층에서 두 칸씩 건너 뛰며 풀링커널 적용. 
    * 상위층의 뉴런 수가 1/4로 줄어듦.
* 패딩 없음
    * 보폭에 따라 일부 행과 열이 무시될 수 있음.

* 최대 풀링층은 아래와 같이 만들 수 있음.

```python
tf.keras.layers.MaxPool2D(
    pool_size=(2, 2), strides=None, padding='valid', data_format=None, **kwargs
)
```

#### 특징

* 파라미터 수를 획기적으로 줄여 계산량, 메모리 사용량을 줄어줌.

* 많은 정보를 잃게 됨. 하지만 그래도 잘 작동함.

* 작은 변화에 대한 어느 정도의 불변성 보장됨(아래 그림 참조).

<img src="images/ch14/homl14-11.png" width="400"/>

* 하지만: 시맨틱 분할의 경우에는 불변성 대신에 등변성이 중요하기도 함.
    * 픽셀이 속한 

### 평균 풀링층

* 풀링커널 구역내의 평균값 활용

* `MaxPool2D` 대신에 `AvgPool2D` 사용함.

* 하지만 최대 풀링층에 비해 성능 떨어짐.
    * 최대값을 유지하여 보다 강한 특성이 학습에 사용되기 때문임.

### 전역평균 풀링층

* 각 특성지도의 평균 계산

```python
    global_avg_pool = keras.layers.GlobalAvgPool2D()
```

또는 

```python
    global_avg_pool = keras.layers.Lambda(lambda X: tf.reduce_mean(X, axis=[1, 2]))
```

* 입력에 사용되는 특성지도의 공간차원(너비와 높이)이 동일한 풀링커널 사용

* 샘플의 각 특성지도에 대해 하나의 숫자 출력

* 출력층에서 유용하게 활용될 수 있음. 

* 현대 신경망 구조에서 종종 활용됨.

### 깊이 위주 최대/평균 풀링층

* 각각 특성지도에 대해 공간(너비와 높이) 위주로 최대/평균을 계산하는 것 대신에 몇 개의 특성지도를 관통하면서 최대/평균을 계산하는 풀링층

<img src="images/ch14/homl14-12.png" width="400"/>

#### 활용

* 다양한 특성에 대한 불변성 학습 가능

* 예제: 손글씨 인식을 위해 동일한 패턴을 회전시키는 여러 개의 필터를 사용하는 경우 활용 가능
    * 두께, 밝기, 왜곡, 색상 등 회전에 상관 없는 불변성 확인 가능.

* 케라스에서 지원하지 않지만 쉽게 구현 가능

## CNN 구조

<img src="images/ch14/homl14-03.png" width="500"/>

### 케라스 활용: 패션 MNIST

```python
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",
    input_shape=[28, 28, 1]), keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"), 
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"), 
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"), 
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"), 
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(), keras.layers.Dense(128, activation="relu"), 
    keras.layers.Dropout(0.5), keras.layers.Dense(64, activation="relu"), 
    keras.layers.Dropout(0.5), keras.layers.Dense(10, activation="softmax")
    ])
```